In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid
import pprint
import argparse

from utils import load_question

from graph import DataExtractor
import json


In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [2]:
import re
import fitz  # PyMuPDF
from pdfminer.high_level import extract_text
import unicodedata

from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

def remove_last_section_from_pdf(file_path: str) -> str:
    """
    PDF 파일에서 조건에 따라 특정 섹션 이후를 제외하고 본문 텍스트만 반환합니다.

    Args:
        file_path (str): PDF 파일 경로.

    Returns:
        str: 특정 섹션 제외된 본문 텍스트.
    """
    # pdfminer를 활용해서 텍스트 추출하기
    full_text = extract_text(file_path)

    # Unicode 정규화
    full_text = unicodedata.normalize("NFKD", full_text)

    # 특정 단어가 있는지 확인
    contains_advancedsciencenews = "www.advancedsciencenews.com" in full_text
    contains_chemelectrochem = "www.chemelectrochem.org" in full_text
    contains_materialsviews = "www.MaterialsViews.com" in full_text

    # 조건에 따라 키워드 설정
    if contains_materialsviews:
        keyword = "Acknowledgements"
    elif contains_advancedsciencenews or contains_chemelectrochem:
        keyword = "Conflict of Interest"
    else:
        keyword = "References"

    # 키워드로 시작하는 부분 중 가장 마지막 부분 찾기
    if keyword == "Conflict of Interest":
        keyword_pattern = r"(?i)c[ o]*n[ f]*l[ i]*c[ t]*[\uFB00]*[ o]*f[ i]*n[ t]*e[ r]*e[ s]*t"
    else:
        keyword_pattern = "(?i)" + keyword.replace(" ", r"\s*")

    matches = list(re.finditer(keyword_pattern, full_text))

    if matches:
        # 마지막 매치의 시작 위치를 기준으로 텍스트를 잘라냄
        last_match = matches[-1]
        full_text = full_text[:last_match.start()]

    return full_text


def embedding_file(
    file_folder: str, 
    file_name: str, 
    rag_method: str, 
    chunk_size: int=500, 
    chunk_overlap: int=100, 
    search_k: int=10
) -> VectorStoreRetriever:
    """문서를 청크 단위로 분할하고 임베딩 모델(text-embedding-ada-002)을 통해 임베딩하여 vector store에 저장합니다. 이후 vector store를 기반으로 검색하는 객체를 생성합니다.

    Args:
        file (str): pdf 문서 경로

    Returns:
        VectorStoreRetriever: 검색기
    """
    ## 긴 텍스트를 작은 청크로 나누는 데 사용되는 클래스
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size,         ## 최대 청크 길이 정의
        chunk_overlap=chunk_overlap,      ## 청크 간 겹침 길이 정의
        separators=["\n\n"]     ## 텍스트를 나눌 때 사용할 구분자를 지정 (문단)
    )
    paper_file_path = f"{file_folder}/{file_name}.pdf"
    
    ## ref 제거 전 코드
    if rag_method == "multiagent-rag":
        loader = PyPDFLoader(paper_file_path)
        docs = loader.load_and_split(text_splitter=splitter) 
       
        ## Embedding 생성 및 vector store에 저장
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_documents(
            documents=docs,         ## 벡터 저장소에 추가할 문서 리스트
            embedding=embeddings    ## 사용할 임베딩 함수
        )
    
    ## ref 제거 후 코드
    elif rag_method == "relevance-rag" or rag_method == "ensemble-rag":
        docs = remove_last_section_from_pdf(file_path=paper_file_path)
        docs = splitter.split_text(docs)
        
        ## Embedding 생성 및 vector store에 저장        
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_texts(
            texts=docs,         ## 벡터 저장소에 추가할 문서 리스트
            embedding=embeddings    ## 사용할 임베딩 함수
        )
    
    ## key error
    else:
        raise KeyError(f"Invalid rag_method: {rag_method}")

    ## 검색기로 변환: 현재 벡터 저장소를 기반으로 VectorStoreRetriever 객체를 생성하는 기능을 제공
    retriever = vector_store.as_retriever(
        search_type="similarity",    ## 어떻게 검색할 것인지? default가 유사도
        search_kwargs={"k": search_k}
    )
    
    print(f"##       {file_name} retriever를 생성했습니다.")
    print(f"##          - chunk_size    :{chunk_size}")
    print(f"##          - chunk_overlap :{chunk_overlap}")
    print(f"##          - retrieve_k    :{search_k}")   

    return retriever


In [ ]:
 self, 
        file_folder:str="./data/input_data", 
        file_number:int=1, 
        # db_folder:str="./vectordb", 
        chunk_size: int=500, 
        chunk_overlap: int=100, 
        search_k: int=10,       
        system_prompt:str = None, 
        model_name:str="gpt-4o",
        save_graph_png:bool=False,

In [27]:
retriever = embedding_file(
            file_folder="data/input_data", 
            file_name="paper_022", 
            rag_method="relevance-rag", 
            # db_folder=db_folder
            chunk_size=500, 
            chunk_overlap=100, 
            search_k=10
        )

##       paper_022 retriever를 생성했습니다.
##          - chunk_size    :500
##          - chunk_overlap :100
##          - retrieve_k    :10


In [28]:
a = retriever.invoke("""
  [
    {
      "CAM (Cathode Active Material)": {
        "Stoichiometry information": {
          "Pristine": {},
          "V-0.005": {}, 
          "V-0.01": {}, 
          "V-0.02": {}
        },
        "Commercial NCM used": {
          "Pristine": <null>,
          "V-0.005": null, 
          "V-0.01": null, 
          "V-0.02": null
        },
        "Lithium source": null,
        "Synthesis method": null,
        "Crystallization method": null,
        "Crystallization final temperature": null,
        "Crystallization final duration (hours)": null,
        "Doping": null,
        "Coating": null,
      }
    }
  ]

""")

In [29]:
b = "\n\n".join(doc.page_content for doc in a)


In [30]:
import pprint
pprint.pprint(b)

('Scientific RepoRts  |          (2019) 9:8952  | '
 'https://doi.org/10.1038/s41598-019-45556-7\n'
 '\n'
 '4\n'
 '\n'
 'www.nature.com/scientificreportswww.nature.com/scientificreports/\x0c'
 'Figure 6.  Initial charge-discharge curves at 0.1 C (a) and cycle '
 'performance at 0.5 C (b) of pristine and V-doped \n'
 'NCM samples.\n'
 '\n'
 'Sample\n'
 '\n'
 'Pristine\n'
 '\n'
 'V-0.005\n'
 '\n'
 'V-0.01\n'
 '\n'
 'V-0.02\n'
 '\n'
 'Initial charge \n'
 'capacity (mAh g−1)\n'
 '\n'
 'Initial discharge \n'
 'capacity (mAh g−1)\n'
 '\n'
 'Initial coulombic \n'
 'efficiency (%)\n'
 '\n'
 'Capacity retention \n'
 'after 80 cycles (%)\n'
 '\n'
 '228.3\n'
 '\n'
 '229.5\n'
 '\n'
 '228.6\n'
 '\n'
 '226.1\n'
 '\n'
 '204.6\n'
 '\n'
 '204.4\n'
 '\n'
 '203.9\n'
 '\n'
 '200.9\n'
 '\n'
 '89.6\n'
 '\n'
 '89.1\n'
 '\n'
 '89.2\n'
 '\n'
 '88.8\n'
 '\n'
 '81.7\n'
 '\n'
 '88.1\n'
 '\n'
 '85.5\n'
 '\n'
 '85.2\n'
 '\n'
 'Table 1.  Electrochemical results of pristine and V-doped NCM samples.\n'
 '\n'
 'to the 